# Nuclei of U2OS cells in a chemical screen

- Dataset [BBBC039](https://data.broadinstitute.org/bbbc/BBBC039/) available from the [Broad Bioimage Benchmark Collection](https://data.broadinstitute.org/bbbc/index.html) ([Ljosa et al., Nature Methods, 2012](http://dx.doi.org/10.1038/nmeth.2083)).

In [1]:
%matplotlib inline

import os
from pathlib import Path
import numpy as np
from sklearn.model_selection import train_test_split
import tqdm
from PIL import Image
import shutil
import matplotlib.pyplot as plt
from skimage import exposure
from skimage import morphology

import sys; sys.path.append("../../../")
import maskflow

root_dir = Path("/home/hadim/.data/Neural_Network/Maskflow/U2OS_Nuclei")

data_dir = root_dir / "Data"
data_dir.mkdir(parents=True, exist_ok=True)

config = maskflow.config.load_config("config.yaml")

# Copy config next to data folder
maskflow.config.save_config(config, root_dir / "config.yaml")

## Download Dataset

In [2]:
images_url = "https://data.broadinstitute.org/bbbc/BBBC039/images.zip"
masks_url = "https://data.broadinstitute.org/bbbc/BBBC039/masks.zip"

# Download and exctract dataset in user data dir
images_path = maskflow.archive.open_archive(images_url, appname="maskflow", progressbar=True, erase=False)
images_path = images_path / 'images'
masks_path = maskflow.archive.open_archive(masks_url, appname="maskflow", progressbar=True, erase=False)
masks_path = masks_path / 'masks'

## Build the dataset

In [13]:
training_size = 0.8  # From 0 to 1
png_compression_level = 0  # From 0 to 9

images_paths = list(images_path.iterdir())
len_dataset = len(images_paths)

train_ids, _ = train_test_split(np.arange(0, len_dataset), train_size=training_size)

train_dir = data_dir / "train_dataset"
train_dir.mkdir(parents=True, exist_ok=True)

test_dir = data_dir / "test_dataset"
test_dir.mkdir(parents=True, exist_ok=True)

train_annotations_path = data_dir / "train_annotations.json"
test_annotations_path = data_dir / "test_annotations.json"

train_annotations = maskflow.dataset.get_base_annotations(config['CLASS_NAMES'], supercategory="nuclei")
test_annotations = maskflow.dataset.get_base_annotations(config['CLASS_NAMES'], supercategory="nuclei")

for image_id, fname in tqdm.tqdm_notebook(enumerate(images_paths), total=len(images_paths)):
     
    image = np.array(Image.open(fname))    
    image = exposure.rescale_intensity(image, out_range='uint8')
    image = image.astype("uint8")
    image = np.expand_dims(image, axis=-1)
    image = np.tile(image, [1, 1, 3])
    
    masks = np.array(Image.open(masks_path / fname.with_suffix('.png').name))
    masks = masks[:, :, 0]
    masks = morphology.label(masks)

    masks_list = []
    for pixel in np.unique(masks)[1:]:
        masks_list.append(masks == pixel)
    masks = np.array(masks_list)
    
    if masks.shape[0] == 0:
        print(f"Discard {fname}")
        continue
    
    # Manual crop to make the dataset smaller
    image = image[50:450, 150:550]
    masks = masks[:, 50:450, 150:550]
    
    # We check masks are not empty
    to_keep = np.where(masks.sum(-1).sum(-1) > 0)[0]
    masks = masks[to_keep]
    
    if masks.shape[0] == 0:
        print(f"Discard {fname}")
        continue
        
    class_ids = np.ones(masks.shape[0]).astype('uint32')
    
    basename = f"nucleus_{image_id:04d}.png"
    
    # Get the annotation in the COCO format.
    image_info, image_annotations = maskflow.dataset.get_annotations(image_id, basename, image, masks, class_ids)
    
    if image_id in train_ids:
        image_path = train_dir / basename
        train_annotations["images"].append(image_info)
        train_annotations["annotations"].extend(image_annotations)
    else:
        image_path = test_dir / basename
        test_annotations["images"].append(image_info)
        test_annotations["annotations"].extend(image_annotations)
    
    Image.fromarray(image).save(str(image_path), compress_level=png_compression_level)
    
maskflow.dataset.save_annotations(train_annotations, train_annotations_path)
maskflow.dataset.save_annotations(test_annotations, test_annotations_path)

Discard /home/hadim/.local/share/maskflow/images/images/IXMtest_F13_s7_w13C1B1D8C-293E-454F-B0FD-6C2C3F9F5173.tif
Discard /home/hadim/.local/share/maskflow/images/images/IXMtest_L01_s2_w1E5038251-DBA3-44D0-BC37-E43E2FC8C174.tif
Discard /home/hadim/.local/share/maskflow/images/images/IXMtest_L10_s6_w12D12D64C-2639-4CA8-9BB4-99F92C9B7068.tif



In [ ]:
# Cleanup Raw Data
import shutil
shutil.rmtree(images_path.parent)
shutil.rmtree(masks_path.parent)

## Check the dataset

In [ ]:
config['SOLVER']['IMS_PER_BATCH'] = 3
config['TEST']['IMS_PER_BATCH'] = 3

# Number of batch to load
n = 1

# Load some data
data_loader = maskflow.dataset.get_data_loader(config, data_dir, is_train=True)
some_data = [iter(data_loader).next() for _ in range(n)]

# Retrieve category's names
categories = data_loader.dataset.coco.cats

for batch_image, batch_target, batch_idx in some_data:
    maskflow.viz.batch_display_top_masks(batch_image, batch_target, batch_idx, categories,
                                         basesize=12, limit=2, cmap="PuBu_r",
                                         pixel_mean=config['INPUT']['PIXEL_MEAN'],
                                         pixel_std=config['INPUT']['PIXEL_STD'])

2018-11-09 12:03:44,023:WARNING:maskrcnn_benchmark.data.build: When using more than one image per GPU you may encounter an out-of-memory (OOM) error if your GPU does not have sufficient memory. If this happens, you can reduce SOLVER.IMS_PER_BATCH (for training) or TEST.IMS_PER_BATCH (for inference). For training, you must also adjust the learning rate and schedule length according to the linear scaling rule. See for example: https://github.com/facebookresearch/Detectron/blob/master/configs/getting_started/tutorial_1gpu_e2e_faster_rcnn_R-50-FPN.yaml#L14
